In [ ]:
import sklearn
from numpy import loadtxt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import datetime
from datetime import datetime, date, timedelta
import xgboost
import pandas as pd 
from xgboost import XGBClassifier, XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
import sklearn.metrics as metrics
import numpy as np
#import catboost as cb
from sklearn.ensemble import AdaBoostRegressor
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

In [5]:
data = pd.read_csv("M_GVB_weather_events_all_covidindex_holidays_binary2020.csv")
data["Date_time"] = pd.to_datetime(data["Date_time"])
data['IS_METRO_OPEN'] = np.where((data['Hour'].isin([0,1,2,3,4,5])), 0, 1)



In [ ]:
def addlogs(res, cols):
    """ Log transform feature list"""
    m = res.shape[1]
    for c in cols:
        res = res.assign(newcol=pd.Series(np.log(1.01+res[c])).values)   
        res.columns.values[m] = c + '_log'
        m += 1
    return res

loglist = ["Checked_out_passengers_week_ago","Checked_in_passengers_week_ago", "BezoekersVerwacht", "Expected visitors to arrive", "Expected visitors to leave"]

data = addlogs(data, loglist)

In [ ]:
data.columns
data = data.dropna()

In [ ]:
features = ["Checked_out_passengers_week_ago", "Checked_in_passengers_week_ago", 'BezoekersVerwacht','StringencyIndex',
 'Event starting',
 'Event ending',
 'Temp(F)',
 'Wind(MpH)',
 'RainFall(in)',
 'duration_rain_knmi',
 'avg_wind_speed_knmi',
 'avg_temp_knmi',
 'sum_rain_knmi',
 'global_radiation',
 'C3_Cancel public events_0.0',
 'C3_Cancel public events_1.0',
 'C3_Cancel public events_2.0',
 'weekday_0.0',
 'weekday_1.0',
 'weekday_2.0',
 'weekday_3.0',
 'weekday_4.0',
 'weekday_5.0',
 'weekday_6.0','Month', 'Hour',
 'HOLIDAY_BINARY', 'VACATION_BINARY',  'Expected visitors to arrive',
 'Expected visitors to leave','IS_METRO_OPEN','Checked_in_passengers_BASELINE', 'Checked_out_passengers_BASELINE']


In [ ]:
def get_train_val_test_split(df):
    """
    Create train, validation, and test split for 1-week ahead models. This means that the last week of the data will be used
    as a test set, the second-last will be the validation set, and the rest will be the training set.
    """
      
    set_testing_date = datetime(2021,11,21,23)

    #set_testing_date = df['datetime'].max()
    
    last_week = pd.date_range(set_testing_date  - timedelta(hours=167), set_testing_date, freq = 'H')
    two_weeks_before = pd.date_range(set_testing_date  - timedelta(hours=335), set_testing_date - timedelta(hours=168),freq = 'H')
    df['Date_time'] = pd.to_datetime(df['Date_time'])
    train1 = df[df['Date_time']<two_weeks_before.min()]
    train2 = df[df['Date_time']<=two_weeks_before.max()]
    validation = df[(df['Date_time']>=two_weeks_before.min()) & (df['Date_time']<=two_weeks_before.max())]
    test = df[(df['Date_time']>=last_week.min()) & (df['Date_time']<=last_week.max())]
    
    return [train1, train2, validation, test]


def split_to_X_Y(dataset):
    df = dataset[dataset["Station"] == "Station Zuid"]
    df_y = df[["Checked_in_passengers", "Checked_out_passengers"]]
    df_X = df[df.columns.intersection(features)]

    return [df_y, df_X]

In [ ]:
train1,train2, validation, test = get_train_val_test_split(data)
train_y, train_X = split_to_X_Y(train1)
validation_y, validation_X = split_to_X_Y(validation)
test_y, test_X = split_to_X_Y(test)


print("training1:", list(train1["Date_time"])[0],list(train1["Date_time"])[-1])
print("training2:", list(train2["Date_time"])[0],list(train2["Date_time"])[-1])
print("validating:",list(validation["Date_time"])[0],list(validation["Date_time"])[-1])
print("testing:",list(test["Date_time"])[0],list(test["Date_time"])[-1])

print(len(train1)/15)
print(len(validation)/15)
print(len(test)/15)

In [ ]:
def combine_predictions(y_pred_check_ins,y_pred_check_outs):
    pred_y = []
      for i in range(len(y_pred_check_ins)):
        pred_y.append([y_pred_check_ins[i],y_pred_check_outs[i]])

    return pred_y 

In [ ]:
model1 = MultiOutputRegressor(XGBRegressor(objective='reg:linear',n_estimators=100,learning_rate=0.2)).fit(train_X, train_y)
pred_y = model1.predict(test_X) 

mae = metrics.mean_absolute_error(test_y, pred_y)
mse = metrics.mean_squared_error(test_y, pred_y)
rmse = np.sqrt(mse) # or mse**(0.5)  
r2 = metrics.r2_score(test_y, pred_y)

print("Results of sklearn.metrics:")
print("MAE:",mae)
print("RMSE:", rmse)
print("R-Squared:", r2)
